##**Script for Annual Medoid Compositing and Calculating Indices of Interest**

### **Setup Script**

**Import common python libs**

In [1]:
import os
import sys
import datetime
from datetime import date
from datetime import datetime
from datetime import datetime, timedelta
import math
import csv
import numpy as np                  # to create a sequence for plotting
from scipy.spatial import distance  # for Jensen-Shannon
import matplotlib.pyplot as plt     # for plotting histograms
import pandas as pd                 # for creating histogram dataframe to export to GDrive
from google.colab import drive      # for exporting from distributed machine to GDrive
from google.colab import files

**Install, import, & authenticate earthengine python API**

In [2]:
##reference: https:#developers.google.com/earth-engine/python_install_manual
!pip install 'pyOpenSSL>=0.11'
!pip install earthengine-api
!pip install folium

     |████████████████████████████████| 61kB 1.8MB/s 
     |████████████████████████████████| 2.6MB 7.0MB/s 


In [3]:
##@title set up authentication credentials (earthengine)
!earthengine authenticate

# test 1: should not show any error message with the following command
#!python -c "import ee; ee.Initialize()"
# Import the Earth Engine Python Package
import ee
# Initialize the Earth Engine object, using the authentication credentials.
ee.Initialize()

# test 2: should print the metadata of the test image
# Print the information for an image asset.
image = ee.Image('srtm90_v4')
print(image.getInfo())

Instructions for updating:
non-resource variables are not supported in the long term
Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=cU8Y4LUixxXXcF4Moox084Cu3YYeOW_QjXLK6fSugmM&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/4wGt5iAiYdnWF67xgcDG_yBjQwO8qe0aQ6ocjq8f9u-mRXLkccFMOi4

Successfully saved authorization token.
{'t

##**Import and Test Folium ImageViz** 

### Import data to test visualization

In [ ]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


###Static Map <BR>
ee.Image objects can be displayed to notebook output cells. The IPython.display module contains the Image function, which can display the results of a URL representing an image generated from a call to the Earth Engine getThumbUrl function. The following cell will display a thumbnail of the global elevation model.

In [ ]:
# Import the Image function from the IPython.display module. 
from IPython.display import Image

# Display a thumbnail of global elevation.
Image(url = dem.updateMask(dem.gt(0))
  .getThumbUrl({'min': 0, 'max': 4000, 'dimensions': 512,
                'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))

### Interactive Map <BR>
The [folium](https://python-visualization.github.io/folium/) library can be used to display ee.Image objects on an interactive [Leaflet](https://leafletjs.com/) map. Folium has no default method for handling tiles from Earth Engine, so one must be defined and added to the folium.Map module before use.

The following cells provide an example of adding a method for handing Earth Engine tiles and using it to display an elevation model to a Leaflet map.

In [ ]:
# Import the folium library.
import folium
from folium import plugins

#### Add custom basemaps

In [ ]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

#### Define add_ee_layer() function

In [ ]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

#### Display ee.Image tiles

In [ ]:
# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3, height=500)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)

# Add the elevation model to the map object.
my_map.add_ee_layer(dem.updateMask(dem.gt(0)), vis_params, 'DEM')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)

NameError: ignored

##**Begin medoid compositing—Principe method** 

Import GEE Composite modules from GEE tools <BR>
[See help document here](https://github.com/gee-community/gee_tools/blob/master/notebooks/composite/medoid.ipynb)
<BR>
[See supporting article here](https://www.mdpi.com/2072-4292/5/12/6481/htm)

In [ ]:
!pip install geetools
!pip install --upgrade geetools
!pip install ipygee

Requirement already up-to-date: geetools in /usr/local/lib/python3.6/dist-packages (0.4.13)


In [ ]:
from geetools import tools, composite, cloud_mask, indices

In [ ]:
from ipygee import *

Import geometries for Great Basin and AIM plots

In [ ]:
#GBbounds = ee.FeatureCollection('users/ericjensen41_default/Thesis/Project_Boundaries/LIIIGBBoundary')
GBbounds = ee.FeatureCollection('users/ericjensen41_default/Thesis/Project_Boundaries/NorCalWinne_GB')
AIMplots = ee.FeatureCollection('users/ericjensen41_default/Thesis/Plots/AIM_modelbuff_100m')
print(AIMplots.first().getInfo())
print(GBbounds.getInfo())

{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-117.77100344785721, 42.33769301660359], [-117.77099449914212, 42.337648390820725], [-117.77098561019372, 42.33760382157669], [-117.77097676035179, 42.33755476864474], [-117.7709588689012, 42.33751017856241], [-117.77094106879544, 42.3374655645895], [-117.77092317741756, 42.337420974600846], [-117.77089643463403, 42.33737639625998], [-117.77086974411652, 42.33733627773599], [-117.77083847688296, 42.33729613249026], [-117.77080278400787, 42.33725599306898], [-117.77076709120706, 42.33721585367323], [-117.77072702498008, 42.337180179851806], [-117.77068686752261, 42.3371445299803], [-117.77064227689331, 42.337113289170226], [-117.77059769391177, 42.33707764510292], [-117.77054863863765, 42.33705089376011], [-117.77049510560065, 42.33701968844591], [-117.77044610265801, 42.336997396819925], [-117.7703881365501, 42.33697060050257], [-117.77033461679184, 42.33694833854968], [-117.77027666385476, 42.33693048558522], [-117.

Build image collection of Landsat 5, 7, and 8. Then filter to dates of interest and cloud mask <BR>
*See notes for things to change*

In [ ]:
L5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR')
L7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')
L8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

L5_bands = L5.first().bandNames().getInfo()
print(L5_bands)
L7_bands = L7.first().bandNames().getInfo()
print(L7_bands)
L8_bands = L8.first().bandNames().getInfo()
print(L8_bands)

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'sr_atmos_opacity', 'sr_cloud_qa', 'pixel_qa', 'radsat_qa']
['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'sr_atmos_opacity', 'sr_cloud_qa', 'pixel_qa', 'radsat_qa']
['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11', 'sr_aerosol', 'pixel_qa', 'radsat_qa']


####Mask Clouds, Water, and Snow

In [ ]:
# def maskL578(image):
#   fillBitMask = 1
#   clearBitMask = 1 << 1
#   waterBitMask = 1 << 2
#   snowBitMask = 1 << 4
#   cloudBitMask = 1 << 5
  
#   ls_bands = ee.List(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa'])
#   pixelqa_name = 'pixel_qa'
  
#   # Get the pixel QA band
#   pixelqa = image.select(pixelqa_name)
  
#   mask = pixelqa.bitwiseAnd(fillBitMask).eq(0)\
#         .And(pixelqa.bitwiseAnd(clearBitMask).neq(0))\
#         .And(pixelqa.bitwiseAnd(waterBitMask).eq(0))\
#         .And(pixelqa.bitwiseAnd(snowBitMask).eq(0))\
#         .And(pixelqa.bitwiseAnd(cloudBitMask).eq(0))      
              
# # Return the masked image (excluding the PIXELQA and STQA layers)
#   return image.updateMask(mask).select(ls_bands)

# # Run the function for each sensor and combination of sensors
# L5_masked = L5.map(maskL578)
# # print(L5_masked)
# L5_masked_bands = L5_masked.first().bandNames().getInfo()
# print(L5_masked_bands)
# L7_masked = L7.map(maskL578)
# L8_masked = L8.map(maskL578)

['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa']


### Mask, filter, and process Landsat Image Collections

In [ ]:
# Bands to select from image collection below
bands_ee = ee.List(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'ndvi', 'pixel_qa'])

# ##### Add for loop for years 1984-2019
L5 = L5\
    .filterBounds(GBbounds)\
    .filterDate('2017-04-01', '2017-06-15')\
    .map(cloud_mask.landsat457SRPixelQA())\
    .map(lambda img: img.addBands(indices.ndvi(img,'B5', 'B4')))\
    .map(lambda img: img.select(bands_ee))

L7 = L7\
    .filterBounds(GBbounds)\
    .filterDate('2017-04-01', '2017-06-15')\
    .map(cloud_mask.landsat457SRPixelQA())\
    .map(lambda img: img.addBands(indices.ndvi(img,'B5', 'B4')))\
    .map(lambda img: img.select(bands_ee))

L8 = L8\
    .filterBounds(GBbounds)\
    .filterDate('2017-04-01', '2017-06-15')\
    .map(cloud_mask.landsat8SRPixelQA())\
    .map(lambda img: img.addBands(indices.ndvi(img,'B5', 'B4')))\
    .map(lambda img: img.select(bands_ee))

L7_bands = L7.first().bandNames().getInfo()
print(L7_bands)

['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'ndvi', 'pixel_qa']


Merge the Landsat collections into single collection

In [ ]:
L578 = L5.merge(L7).merge(L8)

L578_bands = L578.first().bandNames().getInfo()
print(L578_bands)

l578_img = L578.toBands().bandNames().getInfo()
print(l578_img)

['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'ndvi', 'pixel_qa']
['1_2_LE07_041032_20170414_B2', '1_2_LE07_041032_20170414_B3', '1_2_LE07_041032_20170414_B4', '1_2_LE07_041032_20170414_B5', '1_2_LE07_041032_20170414_B6', '1_2_LE07_041032_20170414_B7', '1_2_LE07_041032_20170414_ndvi', '1_2_LE07_041032_20170414_pixel_qa', '1_2_LE07_041032_20170430_B2', '1_2_LE07_041032_20170430_B3', '1_2_LE07_041032_20170430_B4', '1_2_LE07_041032_20170430_B5', '1_2_LE07_041032_20170430_B6', '1_2_LE07_041032_20170430_B7', '1_2_LE07_041032_20170430_ndvi', '1_2_LE07_041032_20170430_pixel_qa', '1_2_LE07_041032_20170601_B2', '1_2_LE07_041032_20170601_B3', '1_2_LE07_041032_20170601_B4', '1_2_LE07_041032_20170601_B5', '1_2_LE07_041032_20170601_B6', '1_2_LE07_041032_20170601_B7', '1_2_LE07_041032_20170601_ndvi', '1_2_LE07_041032_20170601_pixel_qa', '1_2_LE07_042031_20170405_B2', '1_2_LE07_042031_20170405_B3', '1_2_LE07_042031_20170405_B4', '1_2_LE07_042031_20170405_B5', '1_2_LE07_042031_20170405_B6', '1_2_LE07_042031_20

In [ ]:
eprint(L578.size())

Other simple composites to compare

In [ ]:
mosaic = L578.mosaic()

mosaic_bands = mosaic.bandNames().getInfo()
print(mosaic_bands)

['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'ndvi', 'pixel_qa']


**Medoid compositing**
<BR> Add date band before compositing

In [ ]:
def add_date(img):
    date = tools.date.getDateBand(img)
    return img.addBands(date).copyProperties(date, ['day_since_epoch'])
L578 = L578.map(add_date) 
# If need to delete because of multiple date bands — del(L578)

L578_bands = L578.first().bandNames().getInfo()
print(L578_bands)

['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'ndvi', 'pixel_qa', 'date', 'date_1']


In [ ]:
minval = ee.Number(L578.aggregate_min('day_since_epoch'))

In [ ]:
maxval = ee.Number(L578.aggregate_max('day_since_epoch'))

In [ ]:
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
medoid = composite.medoid(L578, bands=bands)

In [ ]:
medoid = medoid.set('min_date', minval, 'max_date', maxval)

medoid_bands = medoid.bandNames().getInfo()
print(medoid_bands)

['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'ndvi', 'pixel_qa', 'date']


**Medoid without taking in count zero values**

In [ ]:
medoid_no_zeros = composite.medoid(L578, bands=bands, discard_zeros=True)

In [ ]:
medoid_no_zeros = medoid_no_zeros.set('min_date', minval, 'max_date', maxval)

medoid_no_zeros_bandsmin = medoid_no_zeros.select(bands)

medoid_no_zeros_bandsmin_bands = medoid_no_zeros_bandsmin.bandNames().getInfo()
print(medoid_no_zeros_bandsmin_bands)

medoid_no_zeros_bands = medoid_no_zeros.bandNames().getInfo()
print(medoid_no_zeros_bands)

medoid_no_zeros_types = medoid_no_zeros.bandTypes().getInfo()
print(medoid_no_zeros_types)

['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'ndvi', 'pixel_qa', 'date']
{'B2': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'B3': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'B4': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'B5': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'B6': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'B7': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'date': {'type': 'PixelType', 'precision': 'int', 'min': -2147483648, 'max': 2147483647}, 'ndvi': {'type': 'PixelType', 'precision': 'float'}, 'pixel_qa': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}}


Export Medoid to assess

In [ ]:
medoid_exp_task = ee.batch.Export.image.toAsset(image = medoid_no_zeros_bandsmin,
                                                  description = 'Medoid', 
                                                  assetId = 'users/ericjensen41_default/MedoidTest',
                                                  scale = 30,
                                                  region = GBbounds.geometry(),
                                                  maxPixels = 1e13)
medoid_exp_task.start()

### **Try to Show on Map**

In [ ]:
Map = Map()
Map.show()

In [ ]:
p = ee.Geometry.Point(-72, -42)

In [ ]:
vis = {'bands':['B5', 'B6','B4'], 'min':0, 'max':5000}

In [ ]:
Map.addLayer(p)
Map.centerObject(p)

In [ ]:
Map.addLayer(mosaic, vis, 'simply Mosaic')

In [ ]:
Map.addLayer(medoid, vis, 'Medoid')

In [ ]:
Map.addLayer(medoid.select('date').randomVisualizer(), {'bands':['viz-red', 'viz-green', 'viz-blue'], 'min':0, 'max':255}, 'dates')

In [ ]:
Map.addLayer(medoid_no_zeros, vis, 'Medoid without zero values')

In [ ]:
Map.addLayer(medoid_no_zeros.select('date').randomVisualizer(), {'bands':['viz-red', 'viz-green', 'viz-blue'], 'min':0, 'max':255}, 'dates of medoid without zero values')

### **Extract data from images and compute locally to compare** <BR>
Extract medoid values in point

In [ ]:
medoid_values = tools.image.getValue(medoid_no_zeros.select(bands), GBbounds.geometry().centroid(), scale=30, side='client')

In [ ]:
medoid_values

{'B2': 1310, 'B3': 1548, 'B4': 2378, 'B5': 2688, 'B6': 2990, 'B7': 2135}

List of values

In [ ]:
medoid_values_list = [val for _, val in medoid_values.items()]

In [ ]:
medoid_values_list

[1310, 1548, 2378, 2688, 2990, 2135]

Extract values at point in each image of the collection

In [ ]:
L578_values = tools.imagecollection.getValues(L578.select(bands), GBbounds.geometry().centroid(), scale=30, side='client')

Get bandnames

In [ ]:
L578_key_list = []
for _, d in L578_values.items():
    keys = []
    for k, v in d.items():
        keys.append(k)        
    L578_key_list.append(keys)

In [ ]:
L578_key_list

Get values as a list

In [ ]:
L578_values_list = []
for _, d in L578_values.items():
    values = []
    for _, v in d.items():
        if v:
            values.append(v)
        else:
            values.append(0)
    L578_values_list.append(values)

In [ ]:
L578_values_list

[[0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [1771, 2000, 2655, 2694, 2886, 2181],
 [1485, 1719, 2546, 2788, 2995, 2173],
 [1446, 1767, 2548, 2743, 3000, 2151],
 [1663, 2031, 2719, 3062, 3176, 2439],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [1392, 1620, 2376, 2641, 2995, 2038],
 [0, 0, 0, 0, 0, 0],
 [1653, 2010, 2855, 3087, 3034, 2376],
 [1492, 1879, 2512, 2958, 3197, 2350],
 [1387, 1635, 2378, 2642, 2990, 2063],
 [0, 0, 0, 0, 0, 0],
 [1620, 2011, 2852, 3064, 3029, 2330],
 [1489, 1877, 2501, 2930, 3193, 2319],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 

In [ ]:
col_values_list = [] for _, d in col_values.items(): values = [] for _, v in d.items(): if v: values.append(v) else: values.append(0) 
col_values_list.append(values)

SyntaxError: ignored

Define Years and dates to include in landsat image collection

In [ ]:
startYear  = 1984;    # what year do you want to start the time series 
endYear    = 2019;    # what year do you want to end the time series
startDay   = '04-01'; # what is the beginniing of date filter | month-day
endDay     = '06-15'; # what is the end of date filter | month-day

NDVI = 'NDVI';
ftvList1 = ['NDVI'];
NBR = 'NBR';
ftvList2 = ['NBR'];
TCB = 'TCB';
ftvList3 = ['TCB'];
TCG = 'TCG';
ftvList4 = ['TCG'];
TCW = 'TCW';
ftvList5 = ['TCW'];

########################################################################
############## Include SAVI, SATVI, NDMI, etc. #########################

Define the segmentation parameters

In [ ]:
# reference: Kennedy, R. E., Yang, Z., & Cohen, W. B. (2010). Detecting trends in forest disturbance and recovery using yearly Landsat time series: 1. LandTrendr—Temporal segmentation algorithms. Remote Sensing of Environment, 114(12), 2897-2910.
#       https://github.com/eMapR/LT-GEE

###################################################################################
################## Needs to be formatted for Python ###############################

run_params = { 
  maxSegments:            6,
  spikeThreshold:         0.9,
  vertexCountOvershoot:   3,
  preventOneYearRecovery: true,
  recoveryThreshold:      0.25,
  pvalThreshold:          0.05,
  bestModelProportion:    0.75,
  minObservationsNeeded:  6
};

Build annual image collection to assess unfitted data coverage

In [ ]:
annualSRcollection = ltgee.buildSRcollection(startYear, endYear, startDay, endDay, aoi);
annualLTcollection1 = ltgee.buildLTcollection(annualSRcollection, NDVI, ftvList1);
annualLTcollection2 = ltgee.buildLTcollection(annualSRcollection, NBR, ftvList2);
annualLTcollection3 = ltgee.buildLTcollection(annualSRcollection, TCB, ftvList3);
annualLTcollection4 = ltgee.buildLTcollection(annualSRcollection, TCG, ftvList4);
annualLTcollection5 = ltgee.buildLTcollection(annualSRcollection, TCW, ftvList5);

# Turn the annualLT image collection (harmonized, calibrated, cloud masked, mediod composites) into a list to select each image and display each on the map.
# This is a visual method to identify years of concern and see which years LT has to fill in gaps, or check for multiyear gaps causing trend anomolies in an area.
# In this case, TCB is being used to identify gaps in the unfitted data.

Begin calculating indices of interest

In [ ]:
# Create ee.Number list of years of analysis to parse list to bandnames
years = ee.List.sequence(startYear, endYear)

NDVI Image Collection

In [ ]:
# Scale NDVI values to -1 to 1 and add year property to image
var ndviImages = function(images) {
    return ee.Image(images)
      .select('NDVI')
      .toInt16()
};

// Apply function to each item in list using map() function
var ndviStack = annualLTcollection1.map(ndviImages)

// Create list of years of analysis to assign to bandnames
var makeList = function(i){
  return ee.String("ndvi").cat(years.get(i)).slice(0,8)
}

var ndviNames = ee.List.sequence(0,(endYear)-startYear).map(makeList)

// Convert to image for calculating zonal statistics and name list
var ndviImage = ndviStack.toBands().set('Year', startYear).rename(ndviNames)
print(ndviImage)


## **Begin medoid compositing—LandTrendr Method**

**Import geometries for Great Basin and AIM plots and apply buffer function**

In [4]:
GBbounds = ee.FeatureCollection('users/ericjensen41_default/Thesis/Project_Boundaries/GBbounds')
#GBbb = ee.FeatureCollection('users/ericjensen41_default/Thesis/Project_Boundaries/GBboundingbox')
GBbb = ee.Feature(GBbounds.first().geometry().bounds())

AIMplots = ee.FeatureCollection('users/ericjensen41_default/Thesis/Plots/Allplots')

# Buffer function to apply to plots
def buffer100(f):
  return f.buffer(100)

AIMplots = AIMplots.map(buffer100)

print(AIMplots.first().getInfo())
print(GBbounds.getInfo())
print(GBbb.getInfo())

{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-120.12868625825102, 39.81919187231709], [-120.12901350776406, 39.81915605637096], [-120.12931470725938, 39.819051459588714], [-120.1295658805991, 39.81888640816144], [-120.12974703423127, 39.818674040560715], [-120.12984374872453, 39.81843126159827], [-120.12984832643083, 39.81817739669358], [-120.12976040396045, 39.81793265350364], [-120.12958698079721, 39.81771651339021], [-120.12934186187259, 39.81754618076458], [-120.12904455855359, 39.81743521372314], [-120.12871873556884, 39.81739244494162], [-120.12839032748839, 39.817421278687995], [-120.12808547462511, 39.81751941988239], [-120.1278284425685, 39.81767905675846], [-120.12763969086173, 39.81788748259494], [-120.12753424448718, 39.8181281070539], [-120.12752049776961, 39.81838177665972], [-120.12759954594036, 39.81862829934358], [-120.1277650976473, 39.81884805172217], [-120.12800397547097, 39.8190235411745], [-120.1282971646923, 39.819140798358], [-120.128621

**Import Landsat image collections**

In [5]:
L5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR')
L7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')
L8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

L5_bands = L5.first().bandNames().getInfo()
print(L5_bands)
L7_bands = L7.first().bandNames().getInfo()
print(L7_bands)
L8_bands = L8.first().bandNames().getInfo()
print(L8_bands)

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'sr_atmos_opacity', 'sr_cloud_qa', 'pixel_qa', 'radsat_qa']
['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'sr_atmos_opacity', 'sr_cloud_qa', 'pixel_qa', 'radsat_qa']
['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11', 'sr_aerosol', 'pixel_qa', 'radsat_qa']


**Define masking functions for Landsat 5 and 7 and for Landsat 8**

In [6]:
# Keep only clear and low confidence cloud pixels
# Mask fill,  water, cloud shadow, snow, and medium and high cloud confidence clouds
# Link to Document: https://prd-wret.s3-us-west-2.amazonaws.com/assets/palladium/production/atoms/files/LSDS-1370_L4-7_SurfaceReflectance-LEDAPS_ProductGuide-v2.pdf
def maskL57(image):
  fillBitMask = 1
  clearBitMask = 1 << 1
  waterBitMask = 1 << 2
  cloudShadowBitMask = 1 << 3
  snowBitMask = 1 << 4
  #cloudBitMask = 1 << 5
  cloudConfBitMask = 1 << 7
  
  ls_bands = ee.List(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'])
  pixelqa_name = 'pixel_qa'
  
  # Get the pixel QA band
  pixelqa = image.select(pixelqa_name)
  
  mask = pixelqa.bitwiseAnd(fillBitMask).eq(0)\
        .And(pixelqa.bitwiseAnd(clearBitMask).neq(0))\
        .And(pixelqa.bitwiseAnd(waterBitMask).eq(0))\
        .And(pixelqa.bitwiseAnd(cloudShadowBitMask).eq(0))\
        .And(pixelqa.bitwiseAnd(snowBitMask).eq(0))\
        .And(pixelqa.bitwiseAnd(cloudConfBitMask).eq(0))      
              
# Return the masked image (excluding the PIXELQA and STQA layers)
  return image.updateMask(mask).select(ls_bands)

In [7]:
# Keep only clear and low confidence cloud pixels and low confidence cirrus pixels
# Mask fill,  water, cloud shadow, snow, terrain occlusion, medium and high cirrus confidence, and medium and high cloud confidence clouds
# Link to Document: https://prd-wret.s3-us-west-2.amazonaws.com/assets/palladium/production/atoms/files/LSDS-1368_L8_SurfaceReflectanceCode-LASRC_ProductGuide-v2.pdf
def maskL8(image):
  fillBitMask = 1
  clearBitMask = 1 << 1
  waterBitMask = 1 << 2
  cloudShadowBitMask = 1 << 3
  snowBitMask = 1 << 4
  # cloudBitMask = 1 << 5
  cloudConfBitMask = 1 << 7
  cirrusConfBitMask = 1 << 9
  occlusionBitMask = 1 << 10
  
  ls_bands = ee.List(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa'])
  pixelqa_name = 'pixel_qa'
  
  # Get the pixel QA band
  pixelqa = image.select(pixelqa_name)
  
  mask = pixelqa.bitwiseAnd(fillBitMask).eq(0)\
        .And(pixelqa.bitwiseAnd(clearBitMask).neq(0))\
        .And(pixelqa.bitwiseAnd(waterBitMask).eq(0))\
        .And(pixelqa.bitwiseAnd(cloudShadowBitMask).eq(0))\
        .And(pixelqa.bitwiseAnd(snowBitMask).eq(0))\
        .And(pixelqa.bitwiseAnd(cloudConfBitMask).eq(0))\
        .And(pixelqa.bitwiseAnd(cirrusConfBitMask).eq(0))\
        .And(pixelqa.bitwiseAnd(occlusionBitMask).eq(0))     
              
# Return the masked image (excluding the PIXELQA and STQA layers)
  return image.updateMask(mask).select(ls_bands)

Apply masking functions to Landsat Image Collections

In [8]:
L5_masked = L5.map(maskL57)
L7_masked = L7.map(maskL57)

L8_masked = L8.map(maskL8)

**Harmonize Landsat 8 (OLI) to Landsat 5 and 7 (ETM)**

In [9]:
#------ L8 to L7 HARMONIZATION FUNCTION -----
# slope and intercept citation: Roy, D.P., Kovalskyy, V., Zhang, H.K.,
# Vermote, E.F., Yan, L., Kumar, S.S, Egorov, A., 2016, Characterization
# of Landsat-7 to Landsat-8 reflective wavelength and normalized
# difference vegetation index continuity, Remote Sensing of Environment,
# 185, 57-70.(http:#dx.doi.org/10.1016/j.rse.2015.12.024) Table 2 -
# reduced major axis (RMA) regression coefficients

def harmonizationRoy(oli):
    # create an image of slopes per band for L8 TO L7 regression line - David
    # Roy
    slopes = ee.Image.constant([0.9785, 0.9542, 0.9825, 1.0073, 1.0171, 0.9949])
    # create an image of y-intercepts per band for L8 TO L7 regression line -
    # David Roy
    itcp = ee.Image.constant([-0.0095, -0.0016, -0.0022, -0.0021, -0.0030, 0.0029])

    # select OLI bands 2-7 and rename them to match L7 band names
    # ...resample the L8 bands using bicubic
    # ...multiply the y-intercept bands by 10000 to match the scale of the L7 bands then apply the line equation - subtract the intercept and divide by the slope
    # return the image as short to match the type of the other data
    # ...set the output system:time_start metadata to the input image time_start otherwise it is null
    y = (oli.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7'], ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']) .resample('bicubic').subtract(itcp.multiply(10000)).divide(slopes).set('system:time_start', oli.get('system:time_start')))                      
    return y.toShort()

In [10]:
L8_harmonized = L8_masked.map(harmonizationRoy)

L8_harm_bands = L8_harmonized.first().bandNames().getInfo()
print(L8_harm_bands)

['B1', 'B2', 'B3', 'B4', 'B5', 'B7']


**Merge Landsat 5, 7, and 8 image collections into single collection**
<BR>*And select bands for calculating medoid*

In [11]:
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']

L578 = L5_masked.merge(L7_masked).merge(L8_harmonized).select(bands).filterBounds(GBbounds)
L578_bands = L578.first().bandNames().getInfo()
print(L578_bands)
print(len(L578_bands))

['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
6


**Mask Saturated Pixels** <BR>
Pixels with values of 20000 are considered saturated pixels and can be considered invalid data because they are not measuring true reflectance values. Therefore, I am masking them.

In [12]:
def mask_saturated(img):
  mask = img.neq(20000)
  masked = img.updateMask(mask)
  return(masked)

In [ ]:
L578 = L578.map(mask_saturated)

**Calculate Medoid Composites for Each Year**

In [13]:
pip install julian

  Created wheel for julian: filename=julian-0.14-cp36-none-any.whl size=2637 sha256=5c1f552824ac771eba68f637e5011dff1753622b6ec93d7f0a9a15e0fa318f2c
  Stored in directory: /root/.cache/pip/wheels/ce/b7/8a/aa742c5ae0a627cc1a665f14550988c012f8c38fc15c2f80b9
Successfully built julian


In [14]:
# make a medoid composite with equal weight among indices
def medoidMosaic(inCollection, dummyCollection):
    """
    LT expects only a single image per year in a time series, there are lost of ways to
    do best available pixel compositing - we have found that a mediod composite requires little logic
    is robust, and fast

    Medoids are representative objects of a data set or a cluster with a data set whose average
    dissimilarity to all the objects in the cluster is minimal. Medoids are similar in concept to
    means or centroids, but medoids are always members of the data set.
    """

    # fill in missing years with the dummy collection get the number of images
    imageCount = inCollection.toList(1).length()
    # if the number of images in this year is 0, then use the dummy collection, otherwise use the SR collection
    finalCollection = ee.ImageCollection(ee.Algorithms.If(imageCount.gt(0), inCollection, dummyCollection))

    # calculate median across images in collection per band
    # calculate the median of the annual image collection
    median = finalCollection.median()

    # calculate the difference between the median and the observation per image per band
    def get_dif_from_med(img):
        # get the difference between each image/band and the corresponding band
        # median and take to power of 2 to make negatives positive and make
        # greater differences weight more
        diff = ee.Image(img).subtract(median).pow(ee.Image.constant(2))
        # per image in collection, sum the powered difference across the bands
        # - set this as the first band add the SR bands to it - now a 7 band
        # image collection
        # Sum band is constant across every pixel
        return diff.reduce('sum').addBands(img)

    difFromMedian = finalCollection.map(get_dif_from_med)

    # get the medoid by selecting the image pixel with the smallest difference
    # between median and observation per band
    # find the powered difference that is the least - what image object is the
    # closest to the median of the collection - and then subset the SR bands
    # and name them - leave behind the powered difference band
    return ee.ImageCollection(difFromMedian).reduce(ee.Reducer.min(len(bands)+1)).select([1,2,3,4,5,6], inCollection.first().bandNames())

# get medoid by year and assign fake time stamp
def reduce_by_year(year):
    """ 
    Get medoid within annual season and assign julian midpoint as date
    """
    y_Str = ee.Number(year).format('%04d')
    y_cli = y_Str.getInfo()

    # Create start and end date strings
    startDate = str(year)+ '-' + '04-01'
    endDate = str(year)+ '-' + '06-15'

    # Convert date strings to DateTime objects
    startDT = datetime.strptime(startDate, '%Y-%m-%d')
    endDT = datetime.strptime(endDate, '%Y-%m-%d')
    print(startDT)

    # Calculate Julian day from DateTime objects
    import julian
    startJD = julian.to_jd(startDT, fmt='jd')
    endJD = julian.to_jd(endDT, fmt='jd')

    # y2_str = ee.Number(year).add(1).format('%04d')
    bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
    filt = L578.filterDate(ee.String(startDate), ee.String(endDate))
   
    # dummy collection for filling missing years
    dummyCollection = ee.ImageCollection([ee.Image([0]*len(bands)).mask(ee.Image(0))])
   
    # get medoid and apply date
    reduced = medoidMosaic(filt, dummyCollection)
    midpoint = startJD+(endJD-startJD)/2
    year = ee.Date(y_Str)
    reduced = reduced.set({'system:time_start':year.millis()})
    return reduced


# get medoid by year and assign fake time stamp
def reduce_by_year_sum(year):
    """ 
    Get medoid within annual season and assign julian midpoint as date
    """
    y_Str = ee.Number(year).format('%04d')
    y_cli = y_Str.getInfo()

    # Create start and end date strings
    startDate = str(year)+ '-' + '07-01'
    endDate = str(year)+ '-' + '09-15'

    # Convert date strings to DateTime objects
    startDT = datetime.strptime(startDate, '%Y-%m-%d')
    endDT = datetime.strptime(endDate, '%Y-%m-%d')
    print(startDT)

    # Calculate Julian day from DateTime objects
    import julian
    startJD = julian.to_jd(startDT, fmt='jd')
    endJD = julian.to_jd(endDT, fmt='jd')

    # y2_str = ee.Number(year).add(1).format('%04d')
    bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
    filt = L578.filterDate(ee.String(startDate), ee.String(endDate))
   
    # dummy collection for filling missing years
    dummyCollection = ee.ImageCollection([ee.Image([0]*len(bands)).mask(ee.Image(0))])
   
    # get medoid and apply date
    reduced = medoidMosaic(filt, dummyCollection)
    midpoint = startJD+(endJD-startJD)/2
    year = ee.Date(y_Str)
    reduced = reduced.set({'system:time_start':year.millis()})
    return reduced

For loop to create ee.ImageCollection of annual composites

In [15]:
# Create list of years of interest for spring
start_year = 2013
end_year = 2017
years = []
years = list(range(start_year,end_year))
print(years)

# Function to run reduce_by_year function for each year in yearList to produce yearly composites.
# Yearly composites are appended to a python list which is returned at the end of the function
TimeSeriesComps = [] # python list to accumulate items in. Couldn't make it work with ee.List()
for iYr in years:
  print(iYr)
  annual_comp = reduce_by_year(iYr)
  annual_comp_I = ee.Image(annual_comp).set({'Year': ee.Number(iYr)}).clip(GBbounds)
  TimeSeriesComps.append(annual_comp_I) # Append annual composites to the list

TimeSeries_EEList = ee.List(TimeSeriesComps) # Cast the python list as an ee.List
TSL_len = TimeSeries_EEList.length().getInfo()
print(TSL_len) # print the length to assess the output

AnnualComps_IC_orig = ee.ImageCollection(TimeSeries_EEList) # Cast the list as an ee.Image Collection

# Print names of bands for each image
AC_bandnames = AnnualComps_IC_orig.first().bandNames().getInfo()
print(AC_bandnames)

[2013, 2014, 2015, 2016]
2013
2013-04-01 00:00:00
2014
2014-04-01 00:00:00
2015
2015-04-01 00:00:00
2016
2016-04-01 00:00:00
4
['B1', 'B2', 'B3', 'B4', 'B5', 'B7']


In [ ]:
# # Function to run reduce_by_year function for each year in yearList to produce yearly composites.
# # Yearly composites are appended to a python list which is returned at the end of the function
# TimeSeriesSum = [] # python list to accumulate items in. Couldn't make it work with ee.List()
# for iYr in years:
#   print(iYr)
#   annual_sum = reduce_by_year_sum(iYr)
#   annual_sum_I = ee.Image(annual_sum).set({'Year': ee.Number(iYr)}).clip(GBbounds)
#   TimeSeriesSum.append(annual_sum_I) # Append annual composites to the list

# TimeSeries_EEList_sum = ee.List(TimeSeriesSum) # Cast the python list as an ee.List
# TSL_len = TimeSeries_EEList_sum.length().getInfo()
# print(TSL_len) # print the length to assess the output

# AnnualComps_IC_sum_orig = ee.ImageCollection(TimeSeries_EEList_sum) # Cast the list as an ee.Image Collection

# # Print names of bands for each image
# AC_bandnames = AnnualComps_IC_sum_orig.first().bandNames().getInfo()
# print(AC_bandnames)

In [ ]:
# years_ee = ee.List.sequence(start_year, end_year, 1)
# # Calculate summer/spring differenced image collection
# TimeSeriesSprSum = []
# def calc_sprsum(year):
#   annual_spr = ee.Image(AnnualComps_IC_orig.filterMetadata('Year', 'equals', year).first())
#   annual_sum = ee.Image(AnnualComps_IC_sum.filterMetadata('Year', 'equals', year).first())
#   annual_sprsum = annual_spr.subtract(annual_sum)
#   return ee.Image(annual_sprsum).set({'Year': ee.Number(iYr)})

# AnnualComps_IC_sprsum = ee.ImageCollection(years_ee.map(calc_sprsum))


# for iYr in years:
#   print(iYr)
#   annual_spr = ee.Image(AnnualComps_IC_orig.filterMetadata('Year', 'equals', iYr).first())
#   print(annual_spr.bandNames().getInfo())
#   annual_sum = ee.Image(AnnualComps_IC_sum.filterMetadata('Year', 'equals', iYr).first())
#   print(annual_sum.bandNames().getInfo())
#   annual_sprsum = annual_spr.subtract(annual_sum)
#   annual_sprsum_I = ee.Image(annual_sprsum).set({'Year': ee.Number(iYr)})
#   TimeSeriesSprSum.append(annual_sprsum_I) # Append annual composites to the list

# TimeSeries_EEList_sprsum = ee.List(TimeSeriesSprSum) # Cast the python list as an ee.List

# AnnualComps_IC_sprsum = ee.ImageCollection(TimeSeries_EEList_sprsum) # Cast the list as an ee.Image Collection

# # Print names of bands for each image
# AC_bandnames = AnnualComps_IC_sprsum.first().bandNames().getInfo()
# print(AC_bandnames)

#### Visualize composite image bands

In [ ]:
# Set visualization parameters.
B1_params = {
  'min': 0,
  'max': 5000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a folium map object.
B1_map = folium.Map(location=[20, 0], zoom_start=3, height=500)

# Add custom basemaps
basemaps['Google Maps'].add_to(B1_map)
# basemaps['Google Satellite Hybrid'].add_to(B1_map)

# Add B1 band to the map object
B1_map.add_ee_layer(AnnualComps_IC.first().select('B1'), B1_params , 'B1')
B1_map.add_ee_layer(AnnualComps_IC.first().select('B2'), B1_params , 'B2')

# Add a layer control panel to the map.
B1_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(B1_map)

# Display the map.
display(B1_map)

### **Calculate Indices based on Medoid Composites**

Indices of Interest

*   NDVI
*   NDMI
*   NBR (doesn't really make sense in this context, but will for fire severity)
*   RBR (doesn't really make sense in this context, but will for fire severity)
*   SAVI
*   SATVI
*   MSAVI2
*   TCG
*   TCB
*   TCW
*   Spatial heterogeneity
*   Spectral heterogeneity

#### **Calculate and add NDVI to composite image collection** 

In [16]:
def add_ndvi(img):
  ndvi = img.normalizedDifference(['B4', 'B3']).rename('NDVI')
  img_wNDVI = img.addBands(ndvi)
  return(img_wNDVI)

In [17]:
AnnualComps_IC = AnnualComps_IC_orig.map(add_ndvi)
# AnnualComps_IC_sum = AnnualComps_IC_sum_orig.map(add_ndvi)
# NDVI_get = AnnualComps_IC_sum.first().bandNames().getInfo()
# print(NDVI_get)

#### **Calculate and add NBR to composite image collection** 

In [18]:
def add_nbr(img):
  nbr = img.normalizedDifference(['B4', 'B7']).rename('NBR')
  img_wNBR = img.addBands(nbr)
  return(img_wNBR)

In [19]:
AnnualComps_IC = AnnualComps_IC.map(add_nbr)
# AnnualComps_IC_sum = AnnualComps_IC_sum.map(add_nbr)
NBR_get = AnnualComps_IC.first().bandNames().getInfo()
print(NBR_get)

['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'NDVI', 'NBR']


#### **Calculate and add NDMI to composite image collection** 

In [20]:
def add_ndmi(img):
  ndmi = img.normalizedDifference(['B4', 'B5']).rename('NDMI')
  img_wNDMI = img.addBands(ndmi)
  return(img_wNDMI)

In [21]:
AnnualComps_IC = AnnualComps_IC.map(add_ndmi)
# AnnualComps_IC_sum = AnnualComps_IC_sum.map(add_ndmi)
NDMI_get = AnnualComps_IC.first().bandNames().getInfo()
print(NDMI_get)

['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'NDVI', 'NBR', 'NDMI']


#### **Calculate and add tasseled cap indices to composite image collection** 

In [22]:
def add_tc(img):
  b = ee.Image(img).select(["B1", "B2", "B3", "B4", "B5", "B7"]) # select the image bands
  brt_coeffs = ee.Image.constant([0.2043, 0.4158, 0.5524, 0.5741, 0.3124, 0.2303]) # set brt coeffs - make an image object from a list of values - each of list element represents a band
  grn_coeffs = ee.Image.constant([-0.1603, -0.2819, -0.4934, 0.7940, -0.0002, -0.1446]) # set grn coeffs - make an image object from a list of values - each of list element represents a band
  wet_coeffs = ee.Image.constant([0.0315, 0.2021, 0.3102, 0.1594, -0.6806, -0.6109]) # set wet coeffs - make an image object from a list of values - each of list element represents a band
 
  sum = ee.Reducer.sum() # create a sum reducer to be applyed in the next steps of summing the TC-coef-weighted bands
  brightness = b.multiply(brt_coeffs).reduce(sum) # multiply the image bands by the brt coef and then sum the bands
  greenness = b.multiply(grn_coeffs).reduce(sum) # multiply the image bands by the grn coef and then sum the bands
  wetness = b.multiply(wet_coeffs).reduce(sum) # multiply the image bands by the wet coef and then sum the bands
  angle = (greenness.divide(brightness)).atan().multiply(ee.Number(180).divide(math.pi)).multiply(100)
  tc = brightness.addBands(greenness).addBands(wetness).addBands(angle).select([0,1,2,3], ['TCB','TCG','TCW','TCA']).set('system:time_start', img.get('system:time_start'))
  img_wTC = img.addBands(tc)
  return img_wTC

In [23]:
AnnualComps_IC = AnnualComps_IC.map(add_tc)
# AnnualComps_IC_sum = AnnualComps_IC_sum.map(add_tc)
TC_get = AnnualComps_IC.first().bandNames().getInfo()
print(TC_get)

['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'NDVI', 'NBR', 'NDMI', 'TCB', 'TCG', 'TCW', 'TCA']


#### **Soil adjusted vegetation indices**
<BR>See supporting article [here](https://www.hindawi.com/journals/js/2017/1353691/)

#### ***Soil-adjusted vegetation index (SAVI)***
Information [here](https://wiki.landscapetoolbox.org/doku.php/remote_sensing_methods:soil-adjusted_vegetation_index)

In [24]:
def add_savi(img):
  # Scale to reflectance values by Landsat scaling factor
  scale = ee.Number(0.0001)
  nir = img.select('B4').multiply(scale)
  red = img.select('B3').multiply(scale)
  L = ee.Number(.5) #soil brightness correction factor

  # Parse out components of equation and then combine for final calculation
  numer = nir.subtract(red)
  denom = nir.add(red).add(L)
  left = numer.divide(denom)
  right = ee.Number(1).add(L)

  savi = left.multiply(right).rename('SAVI')

  img_wSAVI = img.addBands(savi)
  return(img_wSAVI)

In [25]:
AnnualComps_IC = AnnualComps_IC.map(add_savi)
# AnnualComps_IC_sum = AnnualComps_IC_sum.map(add_savi)
SAVI_get = AnnualComps_IC.first().bandNames().getInfo()
print(SAVI_get)

['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'NDVI', 'NBR', 'NDMI', 'TCB', 'TCG', 'TCW', 'TCA', 'SAVI']


***Modified soil-adjusted vegetation index (MSAVI2)*** <BR>
Information [here](https://wiki.landscapetoolbox.org/doku.php/remote_sensing_methods:modified_soil-adjusted_vegetation_index) <BR>
Eliminates need for soil slope calculation as in MSAVI <BR>
[Calculation borrowed from here](https://gis.stackexchange.com/questions/276805/calculate-msavi-modified-soil-adjusted-vegetation-index-in-google-earth-engine)

In [26]:
def add_msavi(img):
  # Scale to reflectance values by Landsat scaling factor
  scale = ee.Number(0.0001)
  nir = img.select('B4').multiply(scale)
  red = img.select('B3').multiply(scale)

  # Calculate MSAVI2
  msavi2 = nir.multiply(2).add(1).subtract(nir.multiply(2).add(1).pow(2).subtract(nir.subtract(red).multiply(8)).sqrt()).divide(2).rename('MSAVI2')
  img_wMSAVI = img.addBands(msavi2)
  return(img_wMSAVI)

In [27]:
AnnualComps_IC = AnnualComps_IC.map(add_msavi)
# AnnualComps_IC_sum = AnnualComps_IC_sum.map(add_msavi)
MSAVI_get = AnnualComps_IC.first().bandNames().getInfo()
print(MSAVI_get)

['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'NDVI', 'NBR', 'NDMI', 'TCB', 'TCG', 'TCW', 'TCA', 'SAVI', 'MSAVI2']


***Soil-adjusted total vegetation index (SATVI)*** <BR>
Information [here](https://wiki.landscapetoolbox.org/doku.php/remote_sensing_methods:soil-adjusted_total_vegetation_index)

In [28]:
def add_satvi(img):
  # Scale to reflectance values by Landsat scaling factor
  scale = ee.Number(0.0001)
  swir2 = img.select('B7').multiply(scale)
  swir1 = img.select('B5').multiply(scale) #requires second nir band, with wavelengths (~1,550 to 1,750nm)
  red = img.select('B3').multiply(scale)
  L = ee.Number(.5) #soil brightness correction factor

  # Parse out components of equation and then combine for final calculation
  Lnumer = swir1.subtract(red)
  Ldenom = swir1.add(red).add(L)
  left = Lnumer.divide(Ldenom)
  right = swir2.divide(ee.Number(2))
  middle = ee.Number(1).add(L)

  satvi = left.multiply(middle).subtract(right).rename('SATVI')

  img_wSATVI = img.addBands(satvi).clip(GBbounds)
  return(img_wSATVI)

In [29]:
AnnualComps_IC = AnnualComps_IC.map(add_satvi)
# AnnualComps_IC_sum = AnnualComps_IC_sum.map(add_satvi).select(['NDVI', 'NBR', 'NDMI', 'TCB', 'TCG', 'TCW', 'TCA', 'SAVI', 'MSAVI2', 'SATVI'])
SATVI_get = AnnualComps_IC.first().bandNames().getInfo()
# SATVI_get_sum = AnnualComps_IC_sum.first().bandNames().getInfo()

# Assess image collection output
SATVI_get_view = AnnualComps_IC.first().propertyNames().getInfo()
SATVI_get_view2 = AnnualComps_IC.first().get('Year').getInfo()
print(SATVI_get)
# print(SATVI_get_sum)
print(SATVI_get_view)
print(SATVI_get_view2)

['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'NDVI', 'NBR', 'NDMI', 'TCB', 'TCG', 'TCW', 'TCA', 'SAVI', 'MSAVI2', 'SATVI']
['system:time_start', 'Year', 'system:index', 'system:bands', 'system:band_names']
2013


### **Unsupervised Classification as measure of spectral heterogeneity**<BR>
Examples would be unsupervised classification classes, etc.<BR>
[Visualization link](https://colab.research.google.com/github/giswqs/earthengine-py-notebooks/blob/master/MachineLearning/clustering.ipynb#scrollTo=6bT1Jn_O3AAf)


In [30]:
def clusterKmeans(img):
  #Get image year
  ImgYr = img.get('Year')

  # Create sampling bounds
  GBgeom = GBbounds.geometry()
  GBfeat = ee.Feature(GBgeom)
  GBbox = GBfeat.bounds().geometry()

    # Make the training dataset.
  training = img.sample(**{
      'region': GBbox, 
      'scale': 30,
      'numPixels': 1000,
      'tileScale': 16})

  # Instantiate the clusterer and train it.
  k25clusterer = ee.Clusterer.wekaKMeans(25).train(training)
  k50clusterer = ee.Clusterer.wekaKMeans(50).train(training)
  k100clusterer = ee.Clusterer.wekaKMeans(100).train(training)
  k200clusterer = ee.Clusterer.wekaKMeans(200).train(training)
  k500clusterer = ee.Clusterer.wekaKMeans(500).train(training)

  # Cluster the input using the trained clusterer.
  kmeans25 = img.cluster(k25clusterer).rename('kmeans25')
  kmeans50 = img.cluster(k50clusterer).rename('kmeans50')
  kmeans100 = img.cluster(k100clusterer).rename('kmeans100')
  kmeans200 = img.cluster(k200clusterer).rename('kmeans200')
  kmeans500 = img.cluster(k500clusterer).rename('kmeans500')
  
  # Add k-means bands
  img_wKmeans = ee.Image(kmeans25.addBands(kmeans50).addBands(kmeans100).addBands(kmeans200)).addBands(kmeans500).set({'Year': ImgYr})
  return(img_wKmeans)

##### **Visualize clustering output**

In [ ]:
GBgeom = GBbounds.geometry()
GBfeat = ee.Feature(GBgeom)
GBbox = GBfeat.bounds().geometry()

# Make the training dataset.
  input = AnnualComps_IC_orig.first()
  
  training = input.sample(**{
      'region': GBbox, 
      'scale': 30, 
      'numPixels': 1500
      })

  # Instantiate the clusterer and train it.
  clusterer = ee.Clusterer.wekaKMeans(100).train(training)
  # k50clusterer = ee.Clusterer.wekaKMeans(50).train(training)
  # k100clusterer = ee.Clusterer.wekaKMeans(100).train(training)
  # k200clusterer = ee.Clusterer.wekaKMeans(200).train(training)

  # Cluster the input using the trained clusterer.
  kmeans = input.cluster(clusterer)
  # kmeans50 = img.cluster(k50clusterer).rename('kmeans50')
  # kmeans100 = img.cluster(k100clusterer).rename('kmeans100')
  
  # Add k-means bands
  #img_wKmeans = input.addBands(kmeans)#.addBands(kmeans50).addBands(kmeans100).addBands(kmeans200)

In [ ]:
import subprocess

try:
    import geehydro
except ImportError:
    print('geehydro package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geehydro'])

import ee
import folium
import geehydro

In [ ]:
Map = folium.Map(location=[40, -100], zoom_start=4)
Map.setOptions('HYBRID')
Map.setCenter(-115, 40, 10)
Map.addLayer(img, {}, 'region')
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map

In [ ]:
Map = folium.Map(location=[40, -100], zoom_start=4)
Map.setOptions('HYBRID')
Map.setCenter(-115, 40, 10)
Map.addLayer(kmeans.randomVisualizer(), {}, 'kmeans')
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map

In [ ]:
imgNames = img_wKmeans.bandNames().getInfo()
print(imgNames)

##### **Run clustering function**

In [ ]:
AnnualComps_IC_clust = AnnualComps_IC_orig.map(clusterKmeans, opt_dropNulls=True)
AnnualComps_IC_clust_first = AnnualComps_IC_clust.first()
AnnualComps_IC_clust_names = AnnualComps_IC_clust_first.bandNames().getInfo()
print(AnnualComps_IC_clust_names)

['kmeans25', 'kmeans50', 'kmeans100', 'kmeans200', 'kmeans500']


**Function to produce zonal statistics of counts of each of the clustering outputs**

In [ ]:
def calc_RR_clust(img):
  # Get image year to filter the AIM plots by
  ImgYr = img.get('Year')
  AIMYr = AIMplots.filterMetadata('VisitYear', 'equals', ImgYr)

  # Combine two reducers to get total cels and count of cells
  reducers = ee.Reducer.count().combine(reducer2 = ee.Reducer.countDistinct(), outputPrefix = 'dist', sharedInputs = True)
  all_reducers = reducers.combine(reducer2 = ee.Reducer.frequencyHistogram(), outputPrefix = '', sharedInputs = True)

  # Run reduce regions with two reducers
  RR_clust = img.reduceRegions(collection = AIMYr, reducer = all_reducers, scale= 30)

  #Function to Normalize the number of unique classes by the total number of cells in the buffered plots
  def normCount(feature):
    DistCount = ee.Number(feature.get('distcount'))
    CellCount = ee.Number(feature.get('count'))
    isNull = ee.Algorithms.IsEqual(DistCount, None)
    return ee.Algorithms.If(isNull, feature.set({'CountNorm': 0}), feature.set({'CountNorm': DistCount.divide(CellCount)}))

  RR_clust_norm = RR_clust.map(normCount)

  return(RR_clust_norm)

**Apply the zonal statistics function to the cluster image collection**

In [ ]:
AIM_zonal_clust = AnnualComps_IC_clust.map(calc_RR_clust)
AIM_zonal_clust_flat = AIM_zonal_clust.flatten()

**Export the cluster zonal statistics to CSV**

In [ ]:
RS_clust_exp_task = ee.batch.Export.table.toDrive(collection = AIM_zonal_clust_flat,
                                                  description = 'RS_clust_Predictors', 
                                                  fileNamePrefix = 'RS_clust_Predictors',
                                                  fileFormat = 'CSV')
RS_clust_exp_task.start()

### **Calculate zonal statistics for each of the bands and remote sensing indices**<BR>
Need to break the AIM data out by year and extract the zonal statistics from the corresponding annual composite

In [ ]:
def calc_RR(img):
  ImgYr = img.get('Year')
  AIMYr = AIMplots.filterMetadata('VisitYear', 'equals', ImgYr)

  RR_mean = img.reduceRegions(collection= AIMYr, reducer= ee.Reducer.mean(), scale= 30).select(['PrimaryKey', 'VisitYear', 'B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'NDVI', 'NBR', 'NDMI', 'TCB', 'TCG', 'TCW', 'TCA', 'SAVI', 'MSAVI2', 'SATVI'],\
                                              ['PrimaryKey', 'VisitYear', 'B1_mn', 'B2_mn', 'B3_mn', 'B4_mn', 'B5_mn', 'B7_mn', 'NDVI_mn', 'NBR_mn', 'NDMI_mn', 'TCB_mn', 'TCG_mn', 'TCW_mn', 'TCA_mn', 'SAVI_mn', 'MSAVI2_mn', 'SATVI_mn'])
  
  RR_stdd = img.reduceRegions(collection= AIMYr, reducer= ee.Reducer.stdDev(), scale= 30).select(['PrimaryKey', 'B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'NDVI', 'NBR', 'NDMI', 'TCB', 'TCG', 'TCW', 'TCA', 'SAVI', 'MSAVI2', 'SATVI'],\
                                              ['PrimaryKey', 'B1_sd', 'B2_sd', 'B3_sd', 'B4_sd', 'B5_sd', 'B7_sd', 'NDVI_sd', 'NBR_sd', 'NDMI_sd', 'TCB_sd', 'TCG_sd', 'TCW_sd', 'TCA_sd', 'SAVI_sd', 'MSAVI2_sd', 'SATVI_sd'])
  
  toyFilter = ee.Filter.equals(
    leftField = 'PrimaryKey',
    rightField = 'PrimaryKey')
  
  innerJoin = ee.Join.inner()
  
  applyJoin = innerJoin.apply(RR_mean, RR_stdd, toyFilter)

  def pairJoin(pair):
    f1 = ee.Feature(pair.get('primary'))
    f2 = ee.Feature(pair.get('secondary'))
    return f1.set(f2.toDictionary())

  joined = applyJoin.map(pairJoin)  
 
  return joined

In [ ]:
def calc_RR_sum(img):
  ImgYr = img.get('Year')
  AIMYr = AIMplots.filterMetadata('VisitYear', 'equals', ImgYr)

  RR = img.reduceRegions(collection= AIMYr, reducer= ee.Reducer.mean(), scale= 30).select(['PrimaryKey', 'VisitYear', 'NDVI', 'NBR', 'NDMI', 'TCB', 'TCG', 'TCW', 'TCA', 'SAVI', 'MSAVI2', 'SATVI'],\
                                              ['PrimaryKey', 'VisitYear', 'NDVI_sum', 'NBR_sum', 'NDMI_sum', 'TCB_sum', 'TCG_sum', 'TCW_sum', 'TCA_sum', 'SAVI_sum', 'MSAVI2_sum', 'SATVI_sum'])
  
  return RR

Run zonal statistics function for each of the annual composites

In [ ]:
AIM_zonal = AnnualComps_IC.map(calc_RR)
AIM_zonal_flat = AIM_zonal.flatten()

In [ ]:
AIM_zonal_sum = AnnualComps_IC_sum.map(calc_RR_sum, opt_dropNulls = True)
AIM_zonal_sum_flat = AIM_zonal_sum.flatten()

Export the zonal statistics of RS indices

In [ ]:
RS_exp_task = ee.batch.Export.table.toDrive(collection = AIM_zonal_flat,
                                                  description = 'RS_Predictors', 
                                                  fileNamePrefix = 'RS_Predictors',
                                                  fileFormat = 'CSV')
RS_exp_task.start()

In [ ]:
RS_exp_task = ee.batch.Export.table.toDrive(collection = AIM_zonal_sum_flat,
                                                  description = 'RS_Predictors_sum', 
                                                  fileNamePrefix = 'RS_Predictors_sum',
                                                  fileFormat = 'CSV')
RS_exp_task.start()

###**Create histograms for calculating bin richness and Shannon's H for each index**

**Function for -1 to 1 indices**

In [ ]:
def norm_histogram(img):
  # Get image year and filter AIM year to match zonal statistics for AIM plots to the correct year
  ImgYr = img.get('Year')
  AIMYr = AIMplots.filterMetadata('VisitYear', 'equals', ImgYr)

  # keep only the -1 to 1 indices
  indices = img.select('NDVI', 'NBR', 'NDMI', 'SAVI', 'MSAVI2', 'SATVI')

  # Create reducer for histograms with 20 and 40 bins
  reducers = ee.Reducer.mean()\
            .combine(reducer2 = ee.Reducer.fixedHistogram(-1, 1, 20), sharedInputs = True, outputPrefix = '20')
  allReducers = reducers\
            .combine(reducer2 = ee.Reducer.fixedHistogram(-1, 1, 40), sharedInputs = True, outputPrefix = '40')

  # Run reduceRegions function for 20 and 40 bins
  hists = ee.FeatureCollection(indices.reduceRegions(collection = AIMYr, reducer = allReducers, scale = 30))

  return(hists)

In [ ]:
NormHists = AnnualComps_IC.map(norm_histogram)
NormHists_flat = NormHists.flatten()

print(AIMplots.size().getInfo())
#print(NormHists_flat.size().getInfo())

11061


Export the normalized histograms

In [ ]:
hist_exp_task = ee.batch.Export.table.toDrive(collection = NormHists_flat,
                                            description = 'NormHists_flat', 
                                            fileNamePrefix = 'NormHists_flat',
                                            fileFormat = 'CSV')
hist_exp_task.start()

**Function for tasseled cap indices**

In [ ]:
# Nested function to calculate tasseled cap histograms for each image
def tc_histograms(img):

  img = ee.Image(img)
 
  # Get image year and filter AIM year to match zonal statistics for AIM plots to the correct year
  ImgYr = img.get('Year')
  ImgStr = ee.String(ImgYr)
  YrStr = str(ImgStr.getInfo())
  
  AIMYr = AIMplots.filterMetadata('VisitYear', 'equals', ImgYr)

  # Need individual images to pass to the function // with single image it is not possible to apply unique reducers
  tcb_img = img.select('TCB')
  tcg_img = img.select('TCG')
  tcw_img = img.select('TCW')
  
  # Inner-function to vary the tasseled cap image, 10th, and 90th percentiles and apply to each below
  def apply_tc_reducer(tc_img, in_fc):

    # Get band name to pass as prefix in reduce regions output
    bandName = tc_img.bandNames()
    bandName = ee.String(bandName.get(0))

    # calculate 10th and 90th percentiles for each index and extract values
    tc_ptiles = tc_img.reduceRegion(reducer = ee.Reducer.percentile([1,99]), geometry = GBbounds.geometry(), scale = 1800, maxPixels = 1e13)

    # Get 90th and 10th percentiles for each band
    tc_p1 = tc_ptiles.get(bandName.cat(ee.String('_p1')))
    tc_p99 = tc_ptiles.get(bandName.cat(ee.String('_p99')))

    # Pass the percentiles to the fixedHistogram functions
    init_reducers = ee.Reducer.mean()\
            .combine(reducer2 = ee.Reducer.fixedHistogram(tc_p1, tc_p99, 20), sharedInputs = True, outputPrefix = bandName.cat(ee.String('_20')))
    all_reducers = init_reducers\
            .combine(reducer2 = ee.Reducer.fixedHistogram(tc_p1, tc_p99, 40), sharedInputs = True, outputPrefix = bandName.cat(ee.String('_40')))

    # Run reduceRegions function for 20 and 40 bins
    hists = ee.FeatureCollection(tc_img.reduceRegions(collection = in_fc, reducer = all_reducers, scale = 30))
    return(hists)

  # Apply the apply_tc_reducer function
  tcb_hists = apply_tc_reducer(tcb_img, AIMYr)
  tcg_hists = apply_tc_reducer(tcg_img, AIMYr)
  tcw_hists = apply_tc_reducer(tcw_img, AIMYr)

  #Export each of the annual histograms
  tcb_exp_task = ee.batch.Export.table.toDrive(collection = tcb_hists,
                                            description = YrStr + '_tcb_hists', 
                                            fileNamePrefix = YrStr + '_tcb_hists',
                                            fileFormat = 'CSV')
  tcb_exp_task.start()

  tcg_exp_task = ee.batch.Export.table.toDrive(collection = tcg_hists,
                                            description = YrStr + '_tcg_hists', 
                                            fileNamePrefix = YrStr + '_tcg_hists',
                                            fileFormat = 'CSV')
  tcg_exp_task.start()
  
  tcw_exp_task = ee.batch.Export.table.toDrive(collection = tcw_hists,
                                            description = YrStr + '_tcw_hists', 
                                            fileNamePrefix = YrStr + '_tcw_hists',
                                            fileFormat = 'CSV')
  tcw_exp_task.start()

Export the tasseled cap histograms

In [ ]:
# Convert to list for looping over index
AnnualComps_List = ee.List(AnnualComps_IC.toList(AnnualComps_IC.size()))
# Get length of list for indexing
CompsList_len = AnnualComps_List.length().getInfo()
# Apply for loop because that's how we do it in GEE (sarcasm)
for i in range(CompsList_len):
  tc_histograms(AnnualComps_List.get(i))

#### Entire LandtrendR algorithm

In [ ]:
######################################################################################################### 
##                                                                                                    #\\
##                                         LANDTRENDR LIBRARY                                         #\\
##                                                                                                    #\\
#########################################################################################################

# date: 2018-06-11
# author: Zhiqiang Yang  | zhiqiang.yang@oregonstate.edu
#         Justin Braaten | jstnbraaten@gmail.com
#         Robert Kennedy | rkennedy@coas.oregonstate.edu
# website: https:#github.com/eMapR/LT-GEE

#########################################################################################################
###### ANNUAL SR TIME SERIES COLLECTION BUILDING FUNCTIONS ##### 
#########################################################################################################

#------ L8 to L7 HARMONIZATION FUNCTION -----
# slope and intercept citation: Roy, D.P., Kovalskyy, V., Zhang, H.K.,
# Vermote, E.F., Yan, L., Kumar, S.S, Egorov, A., 2016, Characterization
# of Landsat-7 to Landsat-8 reflective wavelength and normalized
# difference vegetation index continuity, Remote Sensing of Environment,
# 185, 57-70.(http:#dx.doi.org/10.1016/j.rse.2015.12.024) Table 2 -
# reduced major axis (RMA) regression coefficients

def harmonizationRoy(oli):
    # create an image of slopes per band for L8 TO L7 regression line - David
    # Roy
    slopes = ee.Image.constant([0.9785, 0.9542, 0.9825, 1.0073, 1.0171, 0.9949])
    # create an image of y-intercepts per band for L8 TO L7 regression line -
    # David Roy
    itcp = ee.Image.constant([-0.0095, -0.0016, -0.0022, -0.0021, -0.0030, 0.0029])

    # select OLI bands 2-7 and rename them to match L7 band names
    # ...resample the L8 bands using bicubic
    # ...multiply the y-intercept bands by 10000 to match the scale of the L7 bands then apply the line equation - subtract the intercept and divide by the slope
    # return the image as short to match the type of the other data
    # ...set the output system:time_start metadata to the input image time_start otherwise it is null
    y = (oli.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7'], ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']) .resample('bicubic').subtract(itcp.multiply(10000)).divide(slopes).set('system:time_start', oli.get('system:time_start')))                      
    return y.toShort()

#------ FILTER A COLLECTION FUNCTION -----
def filterCollection(year, startDay, endDay, sensor, aoi):
    return (ee.ImageCollection('LANDSAT/' + sensor + '/C01/T1_SR').filterBounds(aoi).filterDate(str(year) + '-' + str(startDay), str(year) + '-' + str(endDay)))

#------ BUILD A COLLECTION FOR A GIVEN SENSOR AND YEAR -----
def buildSensorYearCollection(year, startDay, endDay, sensor, aoi):
    startMonth = int(startDay[:2])
    endMonth = int(endDay[:2])
    if startMonth > endMonth:
        oldYear = str(int(year) - 1)
        newYear = year
        oldYearStartDay = startDay
        oldYearEndDay = '12-31'
        newYearStartDay = '01-01'
        newYearEndDay = endDay
      
        oldYearCollection = filterCollection(oldYear, oldYearStartDay, oldYearEndDay, sensor, aoi)
        newYearCollection = filterCollection(newYear, newYearStartDay, newYearEndDay, sensor, aoi)

        srCollection = ee.ImageCollection(oldYearCollection.merge(newYearCollection))

    else:
        srCollection = filterCollection(year, startDay, endDay, sensor, aoi)

    return srCollection

#------ RETRIEVE A SENSOR SR COLLECTION FUNCTION -----
def getSRcollection(year, startDay, endDay, sensor, aoi):

    # get a landsat collection for given year, day range, and sensor
    srCollection = buildSensorYearCollection(year, startDay, endDay, sensor, aoi)

    # srCollection = ee.ImageCollection('LANDSAT/'+ strSensor + '/C01/T1_SR')\
    #                     .filterBounds(aoi)\
    #                     .filterDate(year+'-'+startDay, year+'-'+endDay)

    # apply the harmonization function to LC08 (if LC08), subset bands,
    # unmask, and resample
    # condition - if image is OLI
    # true - then apply the L8 TO L7 alignment function after
                # unmasking pixels that were previosuly masked (why/when are
                # pixels masked)
                # false - else select out the reflectance bands from the
                # non-OLI image
                # ...unmask any previously masked pixels
                # ...resample by bicubic
                 # ...set the output system:time_start metadata to the input image time_start otherwise it is null
    def l7_l8_harmonize(img):
        img_cast = ee.String(img.get("SATELLITE"))
        dat = (ee.Image(ee.Algorithms.If(img_cast.compareTo('LANDSAT_8').eq(0),harmonizationRoy(img.unmask()),img.select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7']).unmask().resample('bicubic').set('system:time_start', img.get('system:time_start'))))        )
        return dat
     
    # apply the mask to the image and return it
    def mask_img(img):
        # make a cloud, cloud shadow, and snow mask from fmask band
        # select out the fmask band
        img_cast = ee.Image(img)
        qa = img_cast.select('pixel_qa')
        mask = qa.bitwiseAnd(8).eq(0).And(qa.bitwiseAnd(16).eq(0)).And(qa.bitwiseAnd(32).eq(0))

        # apply the mask - 0's in mask will be excluded from computation and
        # set to opacity=0 in display
        return img_cast.mask(mask)

    srCollection = srCollection.map(l7_l8_harmonize)
    srCollection = srCollection.map(mask_img)

    return srCollection  # return the prepared collection

#------ FUNCTION TO COMBINE LT05, LE07, & LC08 COLLECTIONS -----
def getCombinedSRcollection(year, startDay, endDay, aoi):
    # get TM collection for a given year, date range, and area
    lt5 = getSRcollection(year, startDay, endDay, 'LT05', aoi)
    # get ETM+ collection for a given year, date range, and area
    le7 = getSRcollection(year, startDay, endDay, 'LE07', aoi)
    # get OLI collection for a given year, date range, and area
    lc8 = getSRcollection(year, startDay, endDay, 'LC08', aoi)
    # merge the individual sensor collections into one imageCollection object
    mergedCollection = ee.ImageCollection(lt5.merge(le7).merge(lc8))
    return mergedCollection                                              # return the Imagecollection

#------ FUNCTION TO REDUCE COLLECTION TO SINGLE IMAGE PER YEAR BY MEDOID -----

# make a medoid composite with equal weight among indices
def medoidMosaic(inCollection, dummyCollection):
    """
    LT expects only a single image per year in a time series, there are lost of ways to
    do best available pixel compositing - we have found that a mediod composite requires little logic
    is robust, and fast

    Medoids are representative objects of a data set or a cluster with a data set whose average 
    dissimilarity to all the objects in the cluster is minimal. Medoids are similar in concept to 
    means or centroids, but medoids are always members of the data set.
    """

    # fill in missing years with the dummy collection
    # get the number of images
    imageCount = inCollection.toList(1).length()
    # if the number of images in this year is 0, then use the dummy
    # collection, otherwise use the SR collection
    finalCollection = ee.ImageCollection(ee.Algorithms.If(imageCount.gt(0), inCollection, dummyCollection))

    # calculate median across images in collection per band
    # calculate the median of the annual image collection - returns a
    # single 6 band image - the collection median per band
    median = finalCollection.median()

    # calculate the different between the median and the observation per image
    # per band
    def get_dif_from_med(img):
        # get the difference between each image/band and the corresponding band
        # median and take to power of 2 to make negatives positive and make
        # greater differences weight more
        diff = ee.Image(img).subtract(median).pow(ee.Image.constant(2))
        # per image in collection, sum the powered difference across the bands
        # - set this as the first band add the SR bands to it - now a 7 band
        # image collection
        return diff.reduce('sum').addBands(img)

    difFromMedian = finalCollection.map(get_dif_from_med)

    # get the medoid by selecting the image pixel with the smallest difference
    # between median and observation per band
    # find the powered difference that is the least - what image object is the
    # closest to the median of teh collection - and then subset the SR bands
    # and name them - leave behind the powered difference band
    return ee.ImageCollection(difFromMedian).reduce(ee.Reducer.min(7)).select([1, 2, 3, 4, 5, 6], ['B1', 'B2', 'B3', 'B4', 'B5', 'B7'])

#------ FUNCTION TO APPLY MEDOID COMPOSITING FUNCTION TO A COLLECTION -------------------------------------------
# create a temp iable to hold the upcoming annual mosiac
def buildMosaic(year, startDay, endDay, aoi, dummyCollection):
    # add the year to each medoid image - the data is hard-coded Aug 1st
    from datetime import date
    from time import mktime
    millis = mktime(date(year, 8, 1).timetuple()) * 1000.
    
    # get the SR collection
    collection = getCombinedSRcollection(year, startDay, endDay, aoi)  
    
    # apply the medoidMosaic function to reduce the collection to single image per year by medoid
    img = medoidMosaic(collection, dummyCollection).set('system:time_start', millis)  
    
    return ee.Image(img)

#------ FUNCTION TO BUILD ANNUAL MOSAIC COLLECTION ------------------------------
def buildSRcollection(startYear, endYear, startDay, endDay, aoi):
    # add the year to each image - the data is hard-coded Aug 1st
    from datetime import date
    from time import mktime
    
    # make an image collection from an image with 6 bands all set to 0 and
    # then make them masked values
    dummyCollection = ee.ImageCollection([ee.Image([0, 0, 0, 0, 0, 0]).mask(ee.Image(0))])
    # create empty array to fill
    imgs = ee.List([])
    for i in range(int(startYear), int(endYear)+1):                                           # for each year from hard defined start to end build medoid composite and then add to empty img array
        millis = mktime(date(i, 8, 1).timetuple()) * 1000.
        tmp = buildMosaic(i, startDay, endDay, aoi, dummyCollection)                    # build the medoid mosaic for a given year
        imgs = imgs.cat(tmp.set('system:time_start', millis))       # concatenate the annual image medoid to the collection (img) and set the date of the image - hard coded to the year that is being worked on for Aug 1st
  
    return ee.ImageCollection(imgs)                                                       # return the array img array as an image collection


In [ ]:
test = buildSRcollection('2004', '2005', '04-01', '06-15', GBbounds)
#print(test)
testimg = test.first().bandNames().getInfo()
print(testimg)

EEException: ignored

### **Export layers of interest**

In [32]:
# Year of interest
year_to_export = 2016

# Get year image
AnnualComp_year = AnnualComps_IC.filterMetadata('Year', 'equals', year_to_export).first().select(['TCA', 'B1', 'B3', 'NDVI'])\
                                .reduceNeighborhood(reducer = ee.Reducer.mean(), kernel = ee.Kernel.circle(100, 'meters')).rename(['TCA_mn', 'B1_mn', 'B3_mn', 'NDVI_mn'])

# Get list of bandnames to export over
namelist = AnnualComp_year.bandNames().getInfo();
print(namelist,'namelist');

# Reproject and resample image by nearest neighbors to match Landsat
LS_ref = ee.Image('users/zackrwerner/landsat_harm_reference')
LS_proj = LS_ref.projection() # get projection of landsat harmonized image

for i in namelist:
  # Get image
  img = AnnualComp_year.select(i).clip(GBbounds.geometry());
  
  # Export results to drive
  topo_exp_task = ee.batch.Export.image.toDrive(
      image =  img,
      description = i + '_' + str(year_to_export),
      scale = 30,
      maxPixels = 1e13,
      region = GBbounds.geometry(),
      crs = LS_proj)
  
  topo_exp_task.start()

['TCA_mn', 'B1_mn', 'B3_mn', 'NDVI_mn'] namelist
